<a href="https://colab.research.google.com/github/ubiodee/Plutus_Demo/blob/main/Fine_Tuned_Plutus_Language_Llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning Plutus programming language in Llama3
In this notebook, we will use Llama 3 to train the Plutus programming language.

This Notebook runs in Google Colab and is designed to run on a T4

# Setup the Model
The following section performs all the setup of the model.
This includes

- Installing any dependencies
- Setting any configuration
- Downloading the Base Model

## Install dependencies
In order to get started we need to install the appropriate dependencies

In [ ]:
# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need sentencepiece for the llama2 slow tokenizer
!pip install sentencepiece

# we need einops, used by falcon-7b, llama-2 etc
# einops (einsteinops) is used to simplify tensorops by making them readable
!pip install -q -U einops

# we need to install datasets for our training dataset
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Settings
The following configures our settings for finetuning our model

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Meta-Llama-3-8B"

# The instruction dataset to use
dataset_name = "ubiodee/plutus"

# Fine-tuned model name
new_model = "llama-3-8b-plutus"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 10

## Download the base model
The following will download the base model, in this case the llama-3-7b-chat-hf model.

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# Use 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0},
    offload_folder="offload",  # Optionally specify a folder for offloading


)
model.gradient_checkpointing_enable()
# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

# Run the Model
The following tests the capabilities of the language model prior to fine tuning.

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)



# Run text generation pipeline with our next model
#prompt = "What does the delay keyword stand for in Plutus?"
#prompt = "What is the importance of plutus programming language?"
#prompt = "What is the history of plutus?"
#prompt = "How does staged metaprogramming contribute to the security of Plutus Tx"
#prompt = "Write a hello world program in the plutus programming language, include an explanation of the code "
prompt = "What is the relationship between Plutus Tx and Haskell?. Explain your response to a novice "
#prompt = "Write a Hello Chris program in plutus"
#prompt = "Write a program that checks if a given year, is a leap year in plutus"
#prompt = "Write a hello world program in the Plutus programming language. "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is the relationship between Plutus Tx and Haskell?. Explain your response to a novice  [/INST]  Plutus Tx and Haskell are two distinct programming languages with different functionalities and use cases, although they share some similarities.

Haskell is a purely functional programming language, which means that it emphasizes the use of pure functions, immutability, and the avoidance of side effects. Haskell is statically typed, meaning that the type of every expression is known at compile time, and it has a strong focus on type inference, which means that the programmer does not need to explicitly specify the types of variables or expressions in many cases.

Plutus Tx, on the other hand, is a domain-specific language (DSL) for building smart contracts on the Ethereum blockchain. It is designed to be easy to use and to provide a high-level abstraction over the low-level details of Ethereum smart contract development. Plutus Tx is dynamically typed, meaning that the type o

# Train the Model
The following section is about taking your dataset and then finetuning the model

## Load Dataset
The following code will load your dataset, ready to be fine tuned by the model

In [ ]:
from datasets import load_dataset

# Assuming you've uploaded the dataset files to /content in Colab
dataset = load_dataset('json', data_files={'/content/Plutus_Ubio_Cleansed.json'})

Generating train split: 0 examples [00:00, ? examples/s]

## Fine Tune the Model
The following section will take your dataset, and fine tune the model with it.

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

# Define model and tokenizer identifiers
model_checkpoint = "NousResearch/Meta-Llama-3-8B"  # Replace with actual model identifier
tokenizer_checkpoint = "NousResearch/Meta-Llama-3-8B"  # Replace with actual tokenizer identifier

# Ensure you are authenticated if accessing private models
from huggingface_hub import login

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,  # uses the number of epochs earlier
    per_device_train_batch_size=1,  # 4 seems reasonable
    gradient_accumulation_steps=8,  # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",  # default optimizer
    save_steps=0,  # we're not gonna save
    logging_steps=10,  # same value as used by Meta
    learning_rate=2e-4,  # standard learning rate
    weight_decay=0.001,  # standard weight decay 0.001
    fp16=True,  # set to true for A100
    bf16=False,  # set to true for A100
    max_grad_norm=0.3,  # standard setting
    max_steps=-1,  # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,  # standard warmup ratio
    group_by_length=True,  # speeds up the training
    lr_scheduler_type="cosine",  # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters using SFTConfig
sft_config = SFTConfig(
    output_dir=output_dir,
    dataset_text_field="text",  # field containing text data
    max_seq_length=1024,  # set max sequence length
    packing=False
)

# Load the dataset from the JSONL file
dataset = load_dataset('json', data_files={'/content/Plutus_Ubio_Cleansed.json'})

# Function to format the data into a single "text" field
def format_data(example):
    """Formats the data to have a single "text" field,
    adapting to potential key variations."""
    # Check for different key possibilities
    problem_key = "problem" if "problem" in example else "Problem"
    solution_key = "solution" if "solution" in example else "Solution"
    # Format the text field
    example["text"] = f"### Problem: {example.get(problem_key, '')} ### Solution: {example.get(solution_key, '')}"
    return example

# Apply the formatting function to the dataset
dataset = dataset.map(format_data)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token
# Load the model using `BitsAndBytesConfig` to enable 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Initialize the model with quantization, removing redundant kwargs
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    quantization_config=bnb_config,
    device_map="auto", # Map the model to the first GPU


)
print(dataset)

# Print the first few samples to inspect
print(dataset['train'][0])
# Initialize the SFTTrainer
sft_trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # No max sequence length
    tokenizer=tokenizer,                    # Use the llama tokenizer
    args=training_arguments,                # Use the training arguments
    packing=False,
)

# Train model
sft_trainer.train()

# Save trained model
sft_trainer.model.save_pretrained('new_model')

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Problem', 'Solution', 'response', 'text'],
        num_rows: 316
    })
})
{'Problem': 'Write a Plutus function that adds two integers.', 'Solution': 'addIntegers :: Integer -> Integer -> Integer\naddIntegers a b = a + b', 'response': None, 'text': '### Problem: Write a Plutus function that adds two integers. ### Solution: addIntegers :: Integer -> Integer -> Integer\naddIntegers a b = a + b'}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/316 [00:00<?, ? examples/s]

Step,Training Loss
10,1.574700
20,1.053500
30,0.800900
40,0.861500
50,0.624900
60,0.592600
70,0.610400
80,0.605300
90,0.477300
100,0.447200


# Run the Model
The following runs the model post fine tune

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200,)
prompt = "Write a simple PlutusTx script that always succeeds and passes validation."
result = pipe(prompt)
print(result[0]['generated_text'])


Write a simple PlutusTx script that always succeeds and passes validation. Use the `alwaysSucceeds` function in the PlutusTx module to write this script.
```haskell
{-# LANGUAGE DataKinds           #-}
{-# LANGUAGE NoImplicitPrelude  #-}
{-# LANGUAGE TemplateHaskell    #-}
{-# LANGUAGE OverloadedStrings  #-}
{-# LANGUAGE RecordWildCards    #-}

module AlwaysSucceeds where

import           PlutusTx
import           PlutusTx.Prelude
import           Ledger
import           Ledger.Typed.Scripts

-- | Always succeeds
{-# INLINABLE mkValidator #-}
mkValidator :: () -> ScriptContext -> Bool
mkValidator _ _ = True

validator :: Validator
validator = mkValidatorScript $$(PlutusTx.compile [|| Scripts.wrapValidator mkValidator ||])

scrAddress :: Ledger.Address
scrAddress = scriptAddress validator
```


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [ ]:
# Empty VRAM
del model
del pipe
del sft_trainer # Changed 'trainer' to 'sft_trainer'
import gc
gc.collect()
gc.collect()

23866

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mlabonne/llama-2-7b-miniguanaco/commit/c81a32fd0b4d39e252326e639d63e75aa68c9a4a', commit_message='Upload tokenizer', commit_description='', oid='c81a32fd0b4d39e252326e639d63e75aa68c9a4a', pr_url=None, pr_revision=None, pr_num=None)